# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [ ]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [3]:
function TestsSondabilite_relaxlin(indices_correspond_courant,infaisable,cout,solutions_courant, BestProfit, Bestsol,valeurs_binaires_courant)
    TA, TO, TR = false, false, false
    if(infaisable)#Test de faisabilite
        TA=true
        println("TA")
    elseif(cout <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in (solutions_courant)]-(solutions_courant)) .<= fill(10^-5, size(solutions_courant))) 
        ) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (cout >= BestProfit)
            Bestsol = [v for v in valeurs_binaires_courant]
            #BestProfit=value(benef)
            BestProfit=cout
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_relaxlin (generic function with 1 method)

### Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [4]:

function SeparerNoeud_relaxlin(price_courant,weight_courant,capacity_courant,price_constant_courant,solutions_courant,indices_correspond_courant,list_indice_separation, listvars, listvals,valeurs_binaires_courant)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    #find a fractionnal variable
    i, var = 1, 0
    while((i <= length(solutions_courant)) && (var==0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(solutions_courant[i], digits=0) - solutions_courant[i] ) >= 10^-5)
            var=indices_correspond_courant[i]
            valeurs_binaires_courant[var] = 1.0
            break
        end
        i+=1
    end
    
    #=
    #find most fractionnal variable ?
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#

    push!(listvars,var) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals,1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    push!(list_indice_separation,i)
    
    poids =  weight_courant[i]
    cout = price_courant[i]
    indices_new = deleteat!(indices_correspond_courant,i)
    prices_new = deleteat!(price_courant,i)
    weight_new = deleteat!(weight_courant,i)
    capacity_new = capacity_courant - poids
    price_constant_new = price_constant_courant + cout
    return listvars, listvals,prices_new,weight_new,capacity_new,price_constant_new,indices_new,list_indice_separation,valeurs_binaires_courant
end


function ExplorerAutreNoeud_relaxlin(listvars, listvals, listnodes,price,weight,price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,list_indice_separation,valeurs_binaires_courant)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    capacity_new = capacity_courant
    price_constant_new = price_constant_courant
    #check if we are not at the root node
    if (length(listvars)>= 1)
        #go back to parent node
            var=pop!(listvars)
            theval = pop!(listvals)
            tmp=pop!(listnodes)
            indice_separation = pop!(list_indice_separation)
            indices_new = insert!(indices_correspond_courant, indice_separation, var)
            prices_new = insert!(price_courant, indice_separation, price[var])
            weight_new =  insert!(weight_courant, indice_separation, weight[var])
            capacity_new = capacity_courant + theval*weight[var]
            price_constant_new = price_constant_courant - theval*price[var]

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listvars)>= 1))
            var=pop!(listvars)
            theval = pop!(listvals)
            tmp=pop!(listnodes)
            indice_separation = pop!(list_indice_separation)
            indices_new = insert!(indices_correspond_courant, indice_separation, var)
            prices_new = insert!(price_courant, indice_separation, price[var])
            weight_new =  insert!(weight_courant, indice_separation, weight[var])
            capacity_new = capacity_courant + theval*weight[var]
            price_constant_new = price_constant_courant - theval*price[var]
        end
        if theval==1.0
            indices_new = deleteat!(indices_correspond_courant,indice_separation)
            prices_new = deleteat!(price_courant,indice_separation)
            weight_new = deleteat!(weight_courant,indice_separation)
            valeurs_binaires_courant[var] = 0.0
            push!(list_indice_separation,indice_separation)
            push!(listvars,var)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, listnodes, price_courant,weight_courant,capacity_new,price_constant_new,indices_correspond_courant,list_indice_separation,valeurs_binaires_courant,stop
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [5]:
function optimizer(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,valeurs_binaires_courant)
    stop = false
    R_courant = price_courant./weight_courant
    indicesRcourant = sortperm(R_courant,rev=true)
    i = 1
    j = capacity_courant
    solutions_courant = zeros(length(price_courant))
    cout = price_constant_courant
    if (j < 0 || price_courant == [])
        return true,0,solutions_courant,valeurs_binaires_courant
    end
    while(!stop)
        indice = indicesRcourant[i]
        if (weight_courant[indice] <= j)
            solutions_courant[indice] = 1
            cout = cout + price_courant[indice]
            j = j- weight_courant[indice]
            i = i + 1
        else
            solutions_courant[indice] = j/weight_courant[indice]
            cout = cout + solutions_courant[indice]*price_courant[indice]
            j = 0
            break
        end
        if (i > length(price_courant))
            stop = true
        end
    end
    [valeurs_binaires_courant[indices_correspond_courant[i]] = trunc(Int,solutions_courant[i]) for i in 1:length(indices_correspond_courant)]
    return false,cout,solutions_courant,valeurs_binaires_courant
end


optimizer (generic function with 1 method)

In [6]:
function AfficherNotreModele(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant)
   
            print("Max ")
         if (price_courant != [] )
            print(price_courant[1])
                print(" x[")
                print(indices_correspond_courant[1])
                print("]")
            for i in 2:length(price_courant)
                print(" + ")
                print(price_courant[i])
                print(" x[")
                print(indices_correspond_courant[i])
                print("]")
            end
            print(" + ")
        end
        print(price_constant_courant)
    println("\nSubject to")
        if (weight_courant != [] )
            print(weight_courant[1])
                print(" x[")
                print(indices_correspond_courant[1])
                print("]")
            for i in 2:length(weight_courant)
                print(" + ")
                print(weight_courant[i])
                print(" x[")
                print(indices_correspond_courant[i])
                print("]")
            end
        else
            print("0")
        end
        print(" ≤ ")
        println(capacity_courant)
end

AfficherNotreModele (generic function with 1 method)

In [7]:
function AfficherSolutions(solutions_courant,indices_correspond_courant)
    for i in 1:length(indices_correspond_courant)
        print("x[")
        print(indices_correspond_courant[i])
        print("] = ")
        print(solutions_courant[i])
        print("     ")
    end
end

AfficherSolutions (generic function with 1 method)

### Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [8]:

function SolveKnapInstance(filename)

    if (split(filename,"/")[end] != "test.opb.txt")
        println("This version of the code works only for the test instance !!!!")
    else
        price, weight, capacity = readKnaptxtInstance(filename)
    
        #create the structure to memorize the search tree for visualization at the end
        trParentnodes=Int64[] #will store orig node of arc in search tree
        trChildnodes=Int64[] #will store destination node of arc in search tree
        trNamenodes=[] #will store names of nodes in search tree
         
        #intermediate structure to navigate in the search tree
        listvars=[]
        listvals=[]
        listnodes=[]

        BestProfit=-1
        Bestsol=[]
        valeurs_binaires_courant=zeros(length(price))

        current_node_number=0
        stop = false
        price_courant = [i for i in price]
        weight_courant = [i for i in weight]
        capacity_courant = capacity
        price_constant_courant = 0
        list_indice_separation = []
        indices_correspond_courant = [i for i in 1:length(price)] 

        while(!stop)
            
            println("\nNode number ", current_node_number, ": \n-----\n", )
            AfficherNotreModele(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant)

            #Update the search tree
            push!(trNamenodes,current_node_number+1) 
            if(length(trNamenodes)>=2)
                push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
                push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
            end
            push!(listnodes, current_node_number)


            print("Solve Notre model to compute the bounds of the current node: start ... ")
            infaisable,cout,solutions_courant,valeurs_binaires_courant = optimizer(price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,valeurs_binaires_courant)
            println("... end")

            print("\nSolution relax lin\n"); 
            if(infaisable)#(has_values(model2))
                print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
            else
               AfficherSolutions(solutions_courant,indices_correspond_courant)
            end
            println(" "); println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")

            TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_relaxlin(indices_correspond_courant,infaisable,cout,solutions_courant, BestProfit, Bestsol,valeurs_binaires_courant)
            
            is_node_sondable = TA || TO || TR

            if(!is_node_sondable)
                listvars, listvals,price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,list_indice_separation,valeurs_binaires_courant = SeparerNoeud_relaxlin(price_courant,weight_courant,capacity_courant,price_constant_courant,solutions_courant,indices_correspond_courant,list_indice_separation, listvars, listvals,valeurs_binaires_courant)
            else
                listvars, listvals, listnodes,price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,list_indice_separation,valeurs_binaires_courant, stop = ExplorerAutreNoeud_relaxlin(listvars, listvals, listnodes,price,weight,price_courant,weight_courant,capacity_courant,price_constant_courant,indices_correspond_courant,list_indice_separation,valeurs_binaires_courant)
            end

            current_node_number = current_node_number + 1
        end

        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

        return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes
    end

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [9]:
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)


Node number 0: 
-----

Max 42 x[1] + 40 x[2] + 12 x[3] + 25 x[4] + 0
Subject to
7 x[1] + 4 x[2] + 3 x[3] + 5 x[4] ≤ 10
Solve Notre model to compute the bounds of the current node: start ... ... end

Solution relax lin
x[1] = 0.8571428571428571     x[2] = 1.0     x[3] = 0.0     x[4] = 0.0      

Previous Solution memorized Any[] with bestprofit -1

non sondable

Node number 1: 
-----

Max 40 x[2] + 12 x[3] + 25 x[4] + 42
Subject to
4 x[2] + 3 x[3] + 5 x[4] ≤ 3
Solve Notre model to compute the bounds of the current node: start ... ... end

Solution relax lin
x[2] = 0.75     x[3] = 0.0     x[4] = 0.0      

Previous Solution memorized Any[] with bestprofit -1

non sondable

Node number 2: 
-----

Max 12 x[3] + 25 x[4] + 82
Subject to
3 x[3] + 5 x[4] ≤ -1
Solve Notre model to compute the bounds of the current node: start ... ... end

Solution relax lin
 : NOT AVAILABLE (probably infeasible or ressources limit reached) 

Previous Solution memorized Any[] with bestprofit -1

TA

Node number

UndefVarError: [91mUndefVarError: graphplot not defined[39m